In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null




In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz


In [ ]:
!tar -xvf spark-3.5.1-bin-hadoop3.tgz



In [ ]:
!pip install -q findspark
!pip install pyspark
import findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"


In [6]:
findspark.init()

In [7]:
findspark.__version__

'2.0.1'

In [8]:
from pyspark import SparkContext

In [9]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf


spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkColab") \
    .getOrCreate()

sc =spark.sparkContext


In [10]:
sc

<SparkContext master=local[*] appName=SparkColab>

# temel dataframe işlemleri

In [11]:
spark_dff=spark.read.csv("/content/churn.csv",header=True,inferSchema=True)

In [12]:
spark_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/churn.csv")


In [13]:
spark_dff.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



In [14]:
type(spark_dff)

pyspark.sql.dataframe.DataFrame

In [15]:
spark_dff.cache()

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Churn: int]

In [16]:
spark_dff.show(5,truncate=True)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [17]:
spark_dff.count()

900

# SQL işlemleri

In [18]:
spark_dff.createOrReplaceTempView("tbl_df")
spark_dff.createOrReplaceTempView("tbl_df_2")

In [19]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [20]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   tbl_df|       true|
|         | tbl_df_2|       true|
+---------+---------+-----------+



In [21]:
spark.sql("select * from tbl_df").show(5)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [22]:
spark.sql("select * from tbl_df where Years>6 and Years<7").show(5)

+---+--------------+----+--------------+---------------+-----+---------+-----+
|_c0|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+--------------+----+--------------+---------------+-----+---------+-----+
|  1| Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|   Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3| Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  7| Zachary Walsh|32.0|       9885.12|              1| 6.92|      9.0|    1|
| 11|Bruce Phillips|45.0|       8771.02|              1| 6.64|     11.0|    1|
+---+--------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



# makine öğrenimi

In [23]:
spark_dff=spark_dff.toDF(*[a.lower() for a in spark_dff.columns])
spark_dff.show(5)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           names| age|total_purchase|account_manager|years|num_sites|churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [24]:
spark_dff=spark_dff.withColumnRenamed("_c0","index")
spark_dff.show(5)

+-----+----------------+----+--------------+---------------+-----+---------+-----+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|
+-----+----------------+----+--------------+---------------+-----+---------+-----+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+-----+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [29]:
spark_dff.select("age","total_purchase","account_manager","years","num_sites","churn").describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
names,900,None,None,Aaron King,Zachary Walsh
age,900,41.81666666666667,6.127560416916251,22.0,65.0
total_purchase,900,10062.82403333334,2408.644531858096,100.0,18026.01
account_manager,900,0.4811111111111111,0.4999208935073339,0,1
years,900,5.27315555555555,1.274449013194616,1.0,9.15
num_sites,900,8.587777777777777,1.7648355920350969,3.0,14.0
churn,900,0.16666666666666666,0.3728852122772358,0,1


In [30]:
spark_dff=spark_dff.dropna()

In [31]:
spark_dff=spark_dff.withColumn("age_kare",spark_dff.age**2)
spark_dff.show(5)


+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|  1764.0|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|  1369.0|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
only showing top 5 rows



In [47]:
# bağımlı değişkeni belirtme
spark_dff = spark_dff.drop("label")
from pyspark.ml.feature import StringIndexer
stringIndexer=StringIndexer(inputCol="churn",outputCol="label")
mod=stringIndexer.fit(spark_dff)
indexed=mod.transform(spark_dff)
spark_dff=indexed.withColumn("label",indexed["label"].cast("integer"))

spark_dff.show(5)


+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|label|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|    1|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|    1|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|    1|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|  1764.0|    1|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|  1369.0|    1|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
only showing top 5 rows



In [51]:
#bağımsız değişkenler
from pyspark.ml.feature import VectorAssembler

spark_dff.columns


['index',
 'names',
 'age',
 'total_purchase',
 'account_manager',
 'years',
 'num_sites',
 'churn',
 'age_kare',
 'label']

In [52]:
bagimsizdegiskenler = ['age', 'total_purchase', 'account_manager', 'years', 'num_sites']
vectorAssembler = VectorAssembler(inputCols=bagimsizdegiskenler, outputCol="features")
va_df = vectorAssembler.transform(spark_dff)
va_df.show(5)


+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|label|            features|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|    1|[42.0,11066.8,0.0...|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|    1|[41.0,11916.22,0....|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|    1|[38.0,12884.75,0....|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|  1764.0|    1|[42.0,8010.76,0.0...|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|  1369.0|    1|[37.0,9191.58,0.0...|
+-----+----------------+----+--------------+----

In [54]:
final_df=va_df.select("features","label")
final_df.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
+--------------------+-----+
only showing top 5 rows



In [55]:
#test-train oluşturma
splits=final_df.randomSplit([0.70,0.30])
train_df=splits[0]
test_df=splits[1]

In [56]:
train_df

DataFrame[features: vector, label: int]

In [57]:
test_df

DataFrame[features: vector, label: int]

# GBM ile müşteri terk modellemesi

In [59]:
from pyspark.ml.classification import GBTClassifier
gbm=GBTClassifier(maxIter=10,labelCol="label",featuresCol="features")
gbm_model=gbm.fit(train_df)

In [61]:
y_pred=gbm_model.transform(test_df)

In [62]:
y_pred

DataFrame[features: vector, label: int, rawPrediction: vector, probability: vector, prediction: double]

In [63]:
ac=y_pred.select("label","prediction")

In [64]:
ac.filter(ac.label==ac.prediction).count()/ac.count()

0.8609022556390977

In [72]:
#yeni müşteri terk eder mi etmez mi ?
import pandas as pd
names=pd.Series(["ibrahim","bekir","mine","mustafa","damla"])
age=pd.Series([23,29,30,25,58])
total_purchase=pd.Series([30000,10000,6000,30000,100000])
account_manager=pd.Series([1,0,0,1,1])
Years=pd.Series([20,10,3,8,30])
num_sites=pd.Series([30,8,8,6,50])
yeni_musteriler=pd.DataFrame({"names":names,
                              "age":age,
                              "total_purchase":total_purchase,
                              "account_manager":account_manager,
                              "years":Years,
                              "num_sites":num_sites})
yeni_musteriler.head()

,names,age,total_purchase,account_manager,years,num_sites
0,ibrahim,23,30000,1,20,30
1,bekir,29,10000,0,10,8
2,mine,30,6000,0,3,8
3,mustafa,25,30000,1,8,6
4,damla,58,100000,1,30,50


In [73]:
yeni_sdf=spark.createDataFrame(yeni_musteriler)
yeni_sdf.show()

+-------+---+--------------+---------------+-----+---------+
|  names|age|total_purchase|account_manager|years|num_sites|
+-------+---+--------------+---------------+-----+---------+
|ibrahim| 23|         30000|              1|   20|       30|
|  bekir| 29|         10000|              0|   10|        8|
|   mine| 30|          6000|              0|    3|        8|
|mustafa| 25|         30000|              1|    8|        6|
|  damla| 58|        100000|              1|   30|       50|
+-------+---+--------------+---------------+-----+---------+



In [74]:
yeni_musteriler=vectorAssembler.transform(yeni_sdf)
results=gbm_model.transform(yeni_musteriler)
results.select("names","prediction").show()

+-------+----------+
|  names|prediction|
+-------+----------+
|ibrahim|       1.0|
|  bekir|       0.0|
|   mine|       0.0|
|mustafa|       0.0|
|  damla|       1.0|
+-------+----------+

